In [79]:
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd

- **GDP data**

In [80]:
GDP_df = pd.read_csv('data/A191RL1Q225SBEA.csv')

GDP_df['DATE'] = pd.to_datetime(GDP_df['DATE'])
GDP_df.set_index('DATE', inplace=True)
GDP_df.columns = ['GDP']

- **UMCSI data**

In [81]:
umcsi_df = pd.read_csv('data/UMCSI.csv')

umcsi_df['Date'] = umcsi_df.apply(
    lambda row: "1-{}-{}".format(datetime.strptime(str("%02d" % row['Month']), "%m").strftime("%B"), "%04d" % row['Year']),
    # lambda row: f"1-{row['Month']}-{row['Year']}",
    axis=1
)

umcsi_df['Date'] = pd.to_datetime(umcsi_df['Date'])

umcsi_df.set_index('Date', inplace=True)

umcsi_df.drop(columns=['Month', 'Year'], inplace=True)

In [82]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=umcsi_df.index, 
    y=[74 for _ in range(len(umcsi_df))], 
    name='-',
    line=dict(width=0),
    hoverinfo="skip",
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=umcsi_df.index,
    y=umcsi_df['Index'],
    name='UMCSI',
    fill='tonexty',
    fillcolor='rgba(0, 0, 255, 0.5)',
    line=dict(color='blue', width=1),
))

filtered_gdp_df = GDP_df[GDP_df.index >= umcsi_df.index[0]]

fig.add_trace(go.Bar(
    x=filtered_gdp_df.index,
    y=filtered_gdp_df['GDP'],
    name='GDP',
    marker=dict(color='rgba(0, 0, 0, 0.5)'),
    yaxis="y2"  
))

# ------------------------------------------------------------------------
level_a = 74  # Level on left y-axis
level_b = 0  # Matching level on right y-axis

# Define the ranges for each axis
range_left = [umcsi_df['Index'].min(), umcsi_df['Index'].max()]  # Min and max for right y-axis (to be adjusted)
range_right = [filtered_gdp_df['GDP'].min(), filtered_gdp_df['GDP'].max()]  # Min and max for left y-axis

# Calculate scaling factor to align levels
proportion = (level_a - range_left[0]) / (range_left[1] - range_left[0])  # Position of level_a
range_right[1] = range_right[0] + (level_b - range_right[0]) / proportion  # Adjust right range
# ------------------------------------------------------------------------

fig.update_layout(
    title=dict(text='UMCSI vs GDP'),
    plot_bgcolor='rgb(230, 230,230)',
    width=1400,
    height=700,
    # hovermode="y unified",
    yaxis=dict(title="UMCSI in Points", exponentformat="none", range=range_left, showgrid=True),
    yaxis2=dict(title="Real GDP change in %", overlaying="y", side="right", range=range_right, showgrid=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()